In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from sklearn.preprocessing import KBinsDiscretizer, RobustScaler, MinMaxScaler, OrdinalEncoder,LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import PolynomialFeatures
from mlxtend.feature_extraction import PrincipalComponentAnalysis
from category_encoders import WOEEncoder, TargetEncoder
import matplotlib.pyplot as plt
import seaborn as sns

import gc; gc.enable()

In [ ]:
dfo=pd.read_csv('exercise_05_train.csv')
pd.set_option('display.max_columns', 110)
df=dfo.copy()
df.head()

In [ ]:
numcols = df.select_dtypes(include=['float64', 'int64'])
objcols = df.select_dtypes(include=['object'])

In [1]:
print(len(numcols))
print(len(objcols))

NameError: name 'numcols' is not defined

In [ ]:
df.dtypes.value_counts()

In [2]:
print(df.y.value_counts(normalize=True) * 100)

NameError: name 'df' is not defined

## How wide-ranging are the categorical columns

In [ ]:
print('\n Feature x34 has:',df['x34'].nunique(),'unique values')
print('\n Feature x35 has:',df['x35'].nunique(),'unique values')
print('\n Feature x45 has:',df['x45'].nunique(),'unique values')
print('\n Feature x68 has:',df['x68'].nunique(),'unique values')
print('\n Feature x93 has:',df['x93'].nunique(),'unique values')
print('\n Feature x41 has:',df['x41'].nunique(),'unique values')
print('\n Feature TARGET Y has:',df['y'].nunique(),'unique values')

In [ ]:
print('\nx34 \n',df['x34'].value_counts())
print('\nx35 \n',df['x35'].value_counts())
print('\nx45 \n',df['x45'].value_counts())
print('\nx68 \n',df['x68'].value_counts())
print('\nx93 \n',df['x93'].value_counts())
print('\nx41 \n',df['x41'].value_counts())
print('\nTARGET \n',df['y'].value_counts())

In [ ]:
df.x34 = df.x34.str.lower()

df.x35 = df.x35.replace(['thur','thurday'],'thursday')
df.x35 = df.x35.replace('fri','friday')
df.x35 = df.x35.replace('wed','wednesday')

df.x68 = df.x68.str.lower()
df.x68 = df.x68.str.rstrip('.')

df.x41 = df.x41.str.strip('$')
df.x41 = df.x41.astype(float)
df.x41.dtype

df.x45 = df.x45.str.rstrip('%')
df.x45 = df.x45.astype(float)

In [ ]:
print(df['x41'].min())
print(df['x41'].max())

In [ ]:
# x41: the $ column ranges from -4496.46 to 4327.53
# what does this mean? 
# is this $ balance due? 
# is this insurance loss?

In [ ]:
objcols= [c for c in df.columns.tolist() if c in df.select_dtypes(include=['object'])]
objcols

In [ ]:
df[objcols].describe()

In [ ]:
# show value counts for each column in object type columns in desc order
frames = []
for c in objcols:
    if c != 'x41':
        frame = df[c].value_counts().to_frame().sort_values(by=c, ascending=False)
        frames.append(frame)
    
frames[0].T
frames[1].T

In [ ]:
frames = []
for c in objcols:
    if c != 'x41':
        frame = df[c].value_counts().to_frame().sort_values(by=c, ascending=False)
        frames.append(frame)
    
print(frames[0].T,'\n')
print(frames[1].T,'\n')
print(frames[2].T,'\n')
print(frames[3].T,'\n')
print(frames[4].T,'\n')

## Q - how do i print frames 0-4 in a better way?
## Q - it's 'nicer' to look at WITHOUT the print statement, how?

In [ ]:
for frame in frames:
    print(frame.head().T)

In [ ]:
for frame in frames:
    print(frame.tail().T)

In [ ]:
target = 'y'
feat_cols = [c for c in df.columns.tolist() if c not in [target]] # 100
cnts_cols = [c for c in feat_cols if df[c].dtype == np.float] # 96 x41 and x45 stripped of $ and %, then converted to float

In [ ]:
print(len(feat_cols))
print(len(cnts_cols))

In [ ]:
catcols = df.select_dtypes(include=['object']).copy()
catcols.head()

In [ ]:
gc.collect()

## Baseline Model - Using Only FLOAT Features

In [ ]:
X, y = df[cnts_cols].values, df[target].values

imputer = SimpleImputer()
binner = KBinsDiscretizer(n_bins=2, encode='onehot-dense')
# poly = PolynomialFeatures(interaction_only=True, include_bias=False)
lr = LogisticRegression(class_weight='balanced', n_jobs=-1, random_state=42)
pipe = make_pipeline(imputer, binner, lr) # poly

scores = cross_val_score(pipe, X, y, cv=5, scoring='roc_auc', n_jobs=-1)
scores.mean(), scores.std()

## Using ALL Features

In [ ]:
X, y = df[feat_cols].values, df[target].values

imputer = SimpleImputer(fill_value=None)
binner = KBinsDiscretizer(n_bins=2, encode='onehot-dense')

lr = LogisticRegression(class_weight='balanced', n_jobs=-1, random_state=42)
pipe = make_pipeline(imputer, binner, lr) # poly

scores = cross_val_score(pipe, X, y, cv=5, scoring='roc_auc', n_jobs=-1)
scores.mean(), scores.std()

## PANDAS Practice

In [ ]:
# for col x34, give me the value in index row location 4
df.x34.iloc[4]
df.x34.iloc[[4]] # this syntax works but doesnt return DF, because we only want one field. but this result is different from above result. (includes index #)
# for col x34, give me the values in index row locations 2-4
df.x34.iloc[2:4]
# df.x34.iloc[[2:4]] - this does not work

In [ ]:
# for the whole dataset, pull index row 2
df.iloc[2].head()
# for the whole dataset, pull index row 2, display as dataframe
df.iloc[[2]]
# for the whole dataset, give me row location 2 and 4
df.iloc[[2,4]]
df.loc[[2,4]]

## INVESTIGATE

In [ ]:
df['x41'].max()
# df['x41'].min()
# this returns error: '>=' not supported between instances of 'str' and 'float'
# probably because the column is object type but since we removed the $ and now have numbers, it should be float?? let's try converting it to float below

In [ ]:
# feature x41 look like dollar amounts (continuous), investigating this column
# df.max(axis=0)[x41]
# df.max(axis=0)['x41']
# df['x41'].max()
# df['x41'].min()
df.x41.dtype
# min max isnt working because the values in column are not numeric, will drop the '$' sign then maybe convert column to float?

In [ ]:
## Set all words to lowercase for all columns with text
df.x34=df.x34.apply(lambda x: x.lower())
## DID NOT WORK, not sure how to convert it to string first.

In [ ]:
# what fucking TYPE is x34?
df.x34.dtype

In [ ]:
df.dtypes
## INVESTIGATE: this shows only floats, how do you check for df dtypes that != float64.

In [ ]:
df.x41.dtype
# yes this shows me that x41 is dtype('O') but what if i want to see it specifically mentioned what O means?

In [ ]:
df.select_dtypes(include!=['float64'])
df.dtypes(include != ['float64'])
# df.select_dtypes(include=['object'])

In [ ]:
g= df.columns.to_series().groupby(df.dtypes).groups
{k.name: v for k, v in g.items()}

In [ ]:
# select * columns of a certain type, e.g. object, float, int, bool
df.select_dtypes(include=['object'])

## WHY ARE THESE NOT WORKING???

In [ ]:
%timeit df.x41.str.apply(lambda x: x.replace('$',''))

In [ ]:
%timeit df.x41.apply(lambda x: x.strip('$'))

In [ ]:
%timeit [x.strip('$') for x in df.x41]

## FAIL #1

In [ ]:
# ran for 15+ minutes, showed 'busy'. might be a better idea to run on a smaller dataset like test instead of train. check if it works. aim is check value counts for all object columns but this was checking ALL.
df.apply(pd.Series.value_counts)

In [ ]:
# dfvc = pd.DataFrame(columns=df.columns)
df[objcols].apply(pd.Series.value_counts)
df[['x34', 'x35', 'x41', 'x45', 'x68', 'x93']].apply(pd.Series.value_counts)
# both lines display same thing. this was not displaying anything intelligible so bryan created functions (blocks under)